In [ ]:
from google.colab import drive

In [ ]:
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import math
from sklearn.metrics import r2_score

In [ ]:
data_train=pd.read_csv('/content/drive/MyDrive/ML bootcamp/Copy of linear_train.csv')
data_test=pd.read_csv('/content/drive/MyDrive/ML bootcamp/Copy of linear_test_data (1).csv')

In [ ]:
def normalize_data(x):
 mu=np.mean(x,axis=0)
 sigma=np.std(x,axis=0)
 x=(x-mu)/sigma
 return x

In [ ]:
X=data_train.to_numpy() #CONVERTING DATA TO NUMPY ARRAY
m=X.shape[0]
n=(X.shape[1]-1)
X_Train=X[:,1:n] #APPLYING SLICING ON DATA TO SEPRATE IT FROM LABEL
print(X_Train.shape)
print(X_Train)
X_Train=normalize_data(X_Train) #NORMALIZING DATA / FEATURE SCALING
print(X_Train)
Y_Train=X[:,n] #Y_Train will become a 1d matrix 
Y_Train=Y_Train.reshape(m,1) #but we need a 2 d matrix of order M*1  M IS NO. OF TRAINING EXAMPLES 
print(Y_Train.shape)
print(Y_Train)

(50000, 20)
[[-2.84685568e+02  2.84567363e+02  1.84627063e+01 ... -1.20010842e+01
   8.39780551e+01  7.37317448e+02]
 [-4.28480005e+02  3.07240676e+02  2.26328973e+01 ...  1.80573704e+01
  -5.55216301e+01  7.75407458e+02]
 [-2.58008787e+01  2.37433758e+02 -4.67504670e-01 ...  7.06526251e+01
   3.02121708e+02  8.10402101e+02]
 ...
 [-2.54691431e+02  2.56793278e+02  2.31974534e+01 ... -2.30785214e+01
   1.88422643e+02  7.48559054e+02]
 [ 7.10054262e+02  2.57565009e+02 -1.63356012e+01 ...  1.52388356e+01
   8.95278167e+01  7.57012968e+02]
 [-2.69821407e+02  2.81097270e+02 -3.05824270e+01 ...  8.34763802e+00
   1.44270028e+02  7.32900535e+02]]
[[-0.99722997  1.20760112  0.64714173 ... -0.42356685  0.14706176
  -1.81833211]
 [-1.50051622  2.00042615  0.79297113 ...  0.62858483 -0.831581
  -0.48062945]
 [-0.09112324 -0.44053485 -0.01483786 ...  2.46960382  1.67742154
   0.74836542]
 ...
 [-0.89224929  0.2364157   0.81271337 ... -0.81131613  0.87977983
  -1.42353235]
 [ 2.48439907  0.26340108

In [ ]:
class NeuralNetwork:
    def init_params(architecture):
     w=[]
     b=[]
     for i in range(architecture.shape[1]-1):
         a=np.random.rand(architecture[0][i],architecture[0][i+1])
        # *math.sqrt(2/(architecture[0][i+1]))     
         w.append(a)
     for i in range(architecture.shape[1]-1):
         c=np.random.rand(1,architecture[0][i+1])
        # *math.sqrt(2/(architecture[0][i+1]))
         b.append(c)   

     return w,b



    def sigmoid(input):    
     output = 1 / (1 + np.exp(-input))
     return output



    def sigmoid_derivative(input):
     return NeuralNetwork.sigmoid(input) * (1.0 - NeuralNetwork.sigmoid(input))

      
      
      
    def forward_prop(w,b,X,architecture):
     all_activations=[]
     z_all=[]
     x=X
     for i in range(architecture.shape[1]-2):
       z=np.dot(x,w[i])+b[i]
    # print(z.shape)
       z_all.append(z)
       a=NeuralNetwork.sigmoid(z)
    #  print(a.shape)
       x=a
    #  print(x.shape)
       all_activations.append(a)

     y_predicted=np.dot(x,w[architecture.shape[1]-2])+b[architecture.shape[1]-2]
     z_all.append(y_predicted)
    # print(ypred.shape) 
    # activation = np.exp(ypred) / np.sum(np.exp(ypred),axis=0)
    # print(activation.shape)
     all_activations.append(y_predicted)
    # y_predicted=np.argmax(activation,axis=1,keepdims = True)
    # print(y_predicted.shape)
     return y_predicted,all_activations,z_all





    def backward_prop(X,Y,architecture,w,b,all_activations,z):
      m=X.shape[0]
      derivative_w=[]
      derivative_b=[]
      a=all_activations[-1]-Y
      loss=(1/2*m)*np.sum(np.square(a))

      # lossarray=np.multiply(all_activations[-1],Y)
      # lossarray[np.where(lossarray==0)]=1
      # log=np.log(lossarray)
      # loss=np.sum(log)*(-1/m)

      dz=a
      dW = 1 / m * np.dot(all_activations[-2].T,dz)
      db = 1 / m * np.sum(dz,axis=0,keepdims = True)
      # print(dz.shape)
      # print(dW.shape)
      # print(db.shape)
      derivative_w.append(dW)
      derivative_b.append(db)
      
      for i in range(architecture.shape[1]-3):
        dZi = np.dot(dz,w[architecture.shape[1]-2-i].T) * NeuralNetwork.sigmoid_derivative(z[architecture.shape[1]-3-i])
        # print(dZi.shape)
        dwi= 1 / m * np.dot(all_activations[architecture.shape[1]-4-i].T,dZi)
        # print(dwi.shape)
        dbi= 1 / m * np.sum(dZi,axis=0,keepdims = True)
        # print(dbi.shape)
        derivative_w.append(dwi)
        derivative_b.append(dbi)
        dz=dZi
    #   print(dZi.shape)
    #   print(dwi.shape)
    #   print(dbi.shape)
       
      dz1=np.dot(dz,w[1].T) * NeuralNetwork.sigmoid_derivative(z[0])
      dW1 = 1 / m * np.dot(X.T,dz1)
      db1= 1 / m * np.sum(dz1,axis=0,keepdims = True)
      # print(dz1.shape)
      # print(dW1.shape)       
      # print(db1.shape)
      derivative_w.append(dW1)
      derivative_b.append(db1)
      derivative_w.reverse()
      derivative_b.reverse()
      return derivative_w,derivative_b,loss
 

    def update_params(w,b,dw,db,alpha,a):
      for i in range(a.shape[1]-1):
        w[i] = w[i] - (alpha*dw[i])
        b[i] = b[i] - (alpha*db[i])

      return w, b


    def gradient_descent(X, Y, alpha, iterations,architecture):
      losses=[]
      w,b=NeuralNetwork.init_params(architecture)
      for i in range(iterations):
        y_hat,act,zall=NeuralNetwork.forward_prop(w,b,X,architecture)
        DW,DB,loss=NeuralNetwork.backward_prop(X,Y,architecture,w,b,act,zall)
        w_updated,b_updated=NeuralNetwork.update_params(w,b,DW,DB,alpha,architecture)
        w=w_updated
        b=b_updated
        print("after",i,"iterations")
        print(loss)
        # NeuralNetwork.get_accuracy(y_hat,Y)
        losses.append(loss)

         
      return w,b,losses


    def get_accuracy(y__predicted,y):
        print(r2_score(y,y__predicted))


         
    def accPlot(acc, iters):
      plt.plot(np.arange(iters), acc, '-b') 
      plt.xlabel('Number of iterations')
      plt.ylabel('loss')
      plt.show()

In [ ]:
a=np.array([[20,4,1]])
alpha=0.01
iterations=100000
w_final,b_final,loss_arr=NeuralNetwork.gradient_descent(X_Train,Y_Train,alpha,iterations,a)
# w,b=NeuralNetwork.init_params(a)
# yhat,act,zall=NeuralNetwork.forward_prop(w,b,X_Train,a)
# DW,DB,loss=NeuralNetwork.backward_prop(X_Train,Y_Train,a,w,b,act,zall)


Streaming output truncated to the last 5000 lines.
after 13064 iterations
14661158655714.375
after 13065 iterations
14701833117943.684
after 13066 iterations
14711858619879.734
after 13067 iterations
14783556837023.658
after 13068 iterations
14835577808431.441
after 13069 iterations
14954539247457.402
after 13070 iterations
15075660819626.32
after 13071 iterations
15159754527037.855
after 13072 iterations
15238687806188.861
after 13073 iterations
15163057165012.756
after 13074 iterations
15110909753905.195
after 13075 iterations
14995198989093.078
after 13076 iterations
14904216227376.62
after 13077 iterations
14848886697586.373
after 13078 iterations
14766050755803.078
after 13079 iterations
14746896142071.002
after 13080 iterations
14689986088473.031
after 13081 iterations
14690793886290.967
after 13082 iterations
14655064758135.465
after 13083 iterations
14670451477906.484
after 13084 iterations
14649781802009.594
after 13085 iterations
14678965089186.293
after 13086 iterations
1467

KeyboardInterrupt: ignored